In [ ]:
%pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
base_model = "beomi/Llama-3-Open-Ko-8B-Instruct-preview"

kr_dataset = "/content/drive/MyDrive/korean_commongen/Training/korean_commongen_train_labeled.csv"
kr_validate_dataset = "/content/drive/MyDrive/korean_commongen/Validation/korean_commongen_validate_labeled.csv"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_8bit = False

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_8bit = load_in_8bit,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 1,
    use_rslora = False,
    loftq_config = None,
)

In [ ]:
alpaca_prompt = """아래는 질문 instruction 과 추가정보를 나타내는 input 입니다. 적절한 response를 생성해주세요.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    texts = "<begin_of_text> " + examples["text"] + EOS_TOKEN
    return { "text" : texts, }


EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [ ]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv(kr_dataset)
val_df = pd.read_csv(kr_validate_dataset)

train_dataset = Dataset.from_pandas(df)
valid_dataset = Dataset.from_pandas(val_df)

train_datset = train_dataset.map(formatting_prompts_func, batched=True)
valid_dataset = valid_dataset.map(formatting_prompts_func, batched=True)

print(len(train_datset))
print(len(valid_dataset))

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = valid_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 16,
        warmup_steps = 5,
        max_steps = 100,
        do_eval=True,
        evaluation_strategy="steps",
        logging_steps=1,
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(trainer.state.log_history)

train_loss = df ['loss'].dropna()
val_loss = df ['eval_loss'].dropna()

epochs = range(1, len(train_loss) + 1)

plt.figure(figsize=(10, 5))
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def generate_response(prompt, model):
    prompt = alpaca_prompt.format(prompt, "개념 세트: 카페 이름, 대, 이겼", "")
    messages = [
        {"role": "system", "content": "친절한 챗봇으로서 상대방의 요청에 최대한 자세하고 친절하게 답하자. 모든 대답은 한국어(Korean)으로 대답해줘."},
        {"role": "user", "content": f"{prompt}"},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )

    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)


instruction = "주어진 개념 세트를 이용하여 문장을 생성하세요."

generate_response(instruction, model)

In [ ]:
FINETUNED_MODEL = "drive/MyDrive/llama3_alpaca_Instruct"

trainer.model.save_pretrained(FINETUNED_MODEL)
tokenizer.save_pretrained(FINETUNED_MODEL)